In [2]:
import pandas as pd
import numpy as np
from scipy.integrate import solve_ivp
import json

In [8]:
dat = pd.read_csv('data/targets.csv')
dat = dat[dat.State == 'India']
dat = dat[dat.Year == 2020]

dat = dict(zip(dat.Index, dat.Observed))
dat

{'CNR_Pub': 0.0009442954851492196,
 'CNR_Eng': 0.0003558499973305967,
 'CNR_Acf': 3.876047740795008e-05,
 'PrevA': 0.0017859457437135,
 'PrevS': 0.0005733233270125289,
 'PrevC': 0.0004736356870724853,
 'PrevTxPri': 0.0005805519075276559,
 'OnPriDrugM': 0.000363,
 'OnPriDrugL': 0.000336,
 'OnPriDrugU': 0.000414}

In [38]:
def find_pars_serial(dat, exo):
    adr = exo['adr']
    dur_tx_pri, dur_tx_pub = exo['dur_tx_pri'], exo['dur_tx_pub']
    pr_di_on_txp = exo['pr_di_on_txp']
    ppv_p, ppv_e, ppv_i, ppv_acf = exo['ppv_p'], exo['ppv_e'], exo['ppv_i'], exo['ppv_acf']

    r_sc, r_die = exo['r_sc'], exo['r_die']
    
    cnr_p, cnr_e, cnr_a = dat['CNR_Pub'], dat['CNR_Eng'], dat['CNR_Acf']
    prev_a, prev_s, prev_c = dat['PrevA'], dat['PrevS'], dat['PrevC']
    prev = prev_a + prev_s + prev_c
    pri_drug = dat['OnPriDrugM']

    dur_tx = pri_drug / (cnr_e * pr_di_on_txp)

    # Start calculation
    mu_a, mu_s = r_sc, r_sc + r_die

    r_acf = (cnr_a * ppv_acf) / prev

    det_p = cnr_p * ppv_p
    det_e = cnr_e * ppv_e
    tx_ei = cnr_e * (1 - pr_di_on_txp) / (1 / dur_tx_pri - adr)
    tx_ii = pri_drug - tx_ei
    cnr_i = tx_ii * (1 / dur_tx_pri - adr)
    det_i = (pri_drug - tx_ei) * ppv_i
    det_i = max(det_i, 0)

    det = det_p + det_e + det_i

    p_pub = det_p / det
    p_eng = det_e / (det_e + det_i) 

    r_det = det / prev_c
    r_aware = (r_det + mu_s + r_acf - adr)* prev_c / prev_s
    r_onset = (r_aware + mu_s + r_acf - adr) * prev_s / prev_a
    inc = (r_onset + mu_a + r_acf - adr) * prev_a

    return {
        'inc': inc, 'r_onset': r_onset, 'r_aware': r_aware, 'r_det': r_det,
        'r_die': r_die, 'r_sc': r_sc,
        'p_pub': p_pub, 'p_eng': p_eng,
        'ppv_p': ppv_p, 'ppv_e': ppv_e, 'ppv_i': ppv_i, 'ppv_a': ppv_acf,
        'pr_di_on_txp': pr_di_on_txp,
        'dur_tx_pub': dur_tx_pub, 'dur_tx_pri': dur_tx_pri,
        'adr': adr
    }


In [66]:
exo = {
    'adr': 0.02,
    'dur_tx_pri': 2,
    'dur_tx_pub': 0.5,
    'pr_di_on_txp': 0.7,
    'ppv_p': 1,
    'ppv_e': 1,
    'ppv_i': 1,
    'ppv_acf': 0.5,
    'r_sc': 0.2,
    'r_die': 0.2
}

find_pars_serial(dat, exo)

{'inc': 0.002179434132072903,
 'r_onset': 1.0334838206887247,
 'r_aware': 2.83253954067981,
 'r_det': 3.041872209953855,
 'r_die': 0.2,
 'r_sc': 0.2,
 'p_pub': 0.6554242868991587,
 'p_eng': 0.7167982234606248,
 'ppv_p': 1,
 'ppv_e': 1,
 'ppv_i': 1,
 'ppv_a': 0.5,
 'pr_di_on_txp': 0.7,
 'dur_tx_pub': 0.5,
 'dur_tx_pri': 2,
 'adr': 0.02}